# Data For Teams
> "Better than excel way to do team analytics"

- author: Dmytro Karabash, Maxim Korotkov 
- categories: [management, python]
- image: images/analysis-1841158_1920.jpg
- permalink: /fastreport/

## Introduction

You’re the boss now. You have a team or a business unit doing consultancy. Probably a bunch of Consultants helping clients, some Project Managers leading your projects in a nice Agile or whatever way. Everyone fills their timesheets (and you cut their bonuses if they don’t) – time is billed to clients and you have Fixed Fee projects also. Maybe smaller teams being parts of your team also – you can lead ten, hundred or thousand people (hey, dude – you run a unit of 1000 employees? Don’t read this article – hire someone who did) You probably even have an ERP or something and a CFO in a nice corner office. Do you have a good way to estimate how much profit every team member and project brings and ways to predict it with good precision? If you have a nice ERP dashboard which gives you all then – then you’re lucky and you don’t need this article either. But 70% chances are [a reference to research would be nice - trello] that you get a simple calculation end of year – like “let’s take all revenue finance recognized on your projects in a year and man day cost at $800 to get your unit profitability” ($800 can seem crazily high or unacceptably low, depending on business you’re in). So that’s the precision you have to the money your team brings to firm? Do we need to remind you that this is the only reason why your job exists?

What you probably have is – all the timesheets, projects with their budget and you can even get to approximate costs nicely (we’ll get there). Shitload of data for a year even for a team of 10. What I will tell you here is that you don’t need an ERP to do the rest – the notebook in Jupyter will do for now. Let’s remember - management starts with measurement. How can you supervise your team and projects if you don’t have their data at your fingers?

Why do you need Data Science here? Well, you can have all data an your fingers. Let me show you 3 Project Managers which have highest profit margin from beginning of the year - based on timesheets, revenue type and cost rates in one (not-so-simple) line

In [1]:
#TODO: Dmytro, we need a simple calculation here

In [2]:
#hide

#Note: we might need an internal project to “fill” the difference to full month (e.g. “INTERNAL” with revenue rate of 0 covering everything to full week)

## Data Science

So, let’s get this done. First – yes, you need to know a little bit of python to get the files in. Basic level will do. (If you are a manager in 2020 and can’t write a simple script – hm, you probably missed something). The objective is to have an easy-to-use toolkit. We are definitely not developing any ERP here. What you will see is a very simple set of tools - you spend an hour or two to load your files into the notebook and then start playing with it - just make the analysis you want. You can end up checking a few numbers or building your own charts and dashboards. It is all under L-GPL - free to use, even if you work for a corporation (that’s where you need it most) - just install Anaconda and download the notebook. 
So, we first get and transform our input files (if you want to follow text AND all the code - read it on cohab)

We are loading projects, timesheets and roles below - more details will follow on what we presumed is inside.

In [3]:
#hide
##loading functions

import pandas as pd

#multiple outputs (from ​​​​28 Jupyter Notebook Tips, Tricks, and Shortcuts)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

verbosity=5



In [4]:
#TODO:delete
# data = {}
# for csv_name in ['projects','timesheet', 'roles']:
#     url = 'https://raw.githubusercontent.com/h17/fastreport/master/data/{}.csv'.format(csv_name)
#     data[csv_name] = df = pd.read_csv(url, index_col=0)

In [144]:
data = {}
url = 'https://raw.githubusercontent.com/h17/fastreport/master/data/roles.csv'
data['roles'] = pd.read_csv(url, index_col=0)

project_data = pd.ExcelFile('../data/project_data.xlsm')
project_data.sheet_names

timesheets = []
for sheet in project_data.sheet_names:
    if 'TS' in sheet:
        timesheets += [pd.read_excel(project_data,sheet,header=2)]
    else:
        tmp = pd.read_excel(project_data,sheet)
        data[sheet] = tmp 
if 'Sat.1' in timesheets[0]: timesheets[0] = timesheets[0].rename(columns={'Sat': 'Sun', 'Sat.1' : 'Sat'}) 
data['timesheet'] = pd.concat(timesheets,sort=False)
data['timesheet'] = data['timesheet'][~data['timesheet'].User.isnull()]
data.keys()

['TS Feb', 'TS Mar', 'TS Apr', 'Entity Project', 'HC']

dict_keys(['roles', 'Entity Project', 'HC', 'timesheet'])

In [6]:
data.keys()
for key in data.keys():
    print("{:20} shape: {:^10} rows \t and {:^10} columns".format(key, *data[key].shape))
    #data[key].head(5)

dict_keys(['roles', 'Entity Project', 'HC', 'timesheet'])

roles                shape:     10     rows 	 and     4      columns
Entity Project       shape:     49     rows 	 and     20     columns
HC                   shape:     35     rows 	 and     9      columns
timesheet            shape:    1795    rows 	 and     21     columns


In [7]:
#hide

#todo:separate

## Data we loaded

Let me summarize it here for you – in any decent firm, if you are a team leader or business unit manager you are involved in merit processes – you can get
* Dictionaries
    * Cost per region or default cost for “outside” contributors
    * Average revenue per hour for projects not “owned” by you
    

In [161]:
# show roles
roles=data["roles"]
roles.head(n=5)

,DEFAULT COSTS per ROLE and REGION (in $),EU,EU-UK,EU-CH
0,DIRECTOR,1100.0,1300.0,1100.0
1,Program Manager,900.0,1000.0,900.0
2,Project Manager,800.0,800.0,800.0
3,Project Coordinator,700.0,700.0,700.0
4,Solution Architect,900.0,1000.0,1050.0


* HeadCount
    * Graded costs
    * Engagement model (employee or contractor)

In [156]:
#show head count
headcount=data["HC"]
headcount.head(n=5)

,NAME,TYPE,BU,Location,Position,Level,Management size,ID,START
0,JANE DOE,EMPLOYEE,OPS,CH,CON,SEASONED,NO,1001,TBD
1,JOHN DOE,EMPLOYEE,OPS,PL,CON,JUNIOR,NO,1002,TBD
2,JOHN SMITH,EMPLOYEE,OPS,CH,CON,SEASONED,NO,1003,TBD
3,JACK THE MANAGER,CONTRACTOR,OPS,CH,PRO,SENIOR,NO,1004,TBD
4,JOHN THE MANAGER,CONTRACTOR,OPS,CH,PRO,SENIOR,NO,1005,TBD


* Projects
    * Budget, effort estimate, deadline
    * Project type (e.g. “On demand support” or “Subscription” will not have any actual deadline to control) - though it is ok if you don’t have it
    * Type of revenue recognition (Time and Material or Fixed Fee)


In [9]:
# show projects
projects = data["Entity Project"]
projects.head(n=5)

,Client ID,Project ID,Project,PM,Client Region,Client Country,Category,Recon Status,FIN Status,Start,End,Rev Rec,Funding Method,BASE RATE USD,TCV,Recognized USD,2017 effort,2018 effort,2019 effort,Jan 2020 Effort
0,13154.0,68454.0,Medium consutancy 1,SUSAN THE MANAGER,Europe,France,Consulting,Approved,APPROVED,2019-01-16 08:00:00,2020-07-22 14:00:00,T&M,Billable to Client Variable TM,1200.0,"39,000 EUR",41540.0800,0.0,0.0,180.5,21.50
1,2764.0,70285.0,Our biggest project,EXT 6a8e0ca747,Europe,United Kingdom,Consulting,Approved,APPROVED,2019-10-01 08:00:00,2021-04-30 11:00:00,T&M,Billable to Client Variable TM,1200.0,"1,450,999 USD",473002.0800,0.0,0.0,1241.5,458.25
2,12916.0,68093.0,Upgrade to new version,TIM THE LEAVER,Europe,Netherlands,Consulting,Approved,APPROVED,2018-09-28 08:00:00,2020-04-27 17:00:00,FF,Billable to Client (Fixed Fee),1200.0,"127,500 EUR",142411.5533,0.0,0.0,0.0,0.00
3,12916.0,68094.0,Small change on top of upgrade,JOHN THE MANAGER,Europe,Netherlands,Consulting,Approved,APPROVED,2018-09-28 08:00:00,2020-05-11 17:00:00,T&M,Billable to Client Variable TM,1200.0,"12,500 EUR",0.0000,0.0,0.0,0.0,0.00
4,18933.0,68726.0,Transition from another product,JANE THE MANAGER,Europe,France,Consulting,Approved,APPROVED,2019-03-01 08:00:00,2020-05-27 16:00:00,FF,Billable to Client (Fixed Fee),1200.0,"143,250 EUR",132677.3616,0.0,0.0,525.5,5.00


* Timesheets (your team / your projects, note that other teams can contribute to your projects and your team might work in other projects) 
    * Typically arrive monthly or weekly
(optional) Plans
    * If there are no plans, below we talk about an option to “plan auto-fill”

But your ERP probably does not present you anything nicely in terms of parameters you’d want to control on your level  (otherwise you won’t read this anyway). There is even maybe a project running so it will do so by 2025. 
If you are steering this boat long enough – you might have ended up with some quick and dirty calculation, SQL queries to SAP (with field names still in German) or Excel file. Why? Because, well, your bosses don’t give a shit – they already hired you and a better ERP is coming in 2025 anyway – so they want to know how much money your team makes and why is it not so profitable (because it never is)

In [148]:
# show timesheets

timesheet=data["timesheet"]
timesheet.head(n=5)

,Project,Project Phase,Work Description,Tax/Billing Activity,Activity Type,Billing Flag,User,Roles (Primary),Req Name,Employee Type,...,Effort Type,Timesheet Status,Total (Days),Sun,Mon,Tue,Wed,Thu,Fri,Sat
0,PRJ d41d8cd98f,Core Deployment,Consultancy,Consultancy,NaN,Billable,EXT 800982ab08,OB Developer,NaN,Employee,...,Actual,Progressed,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
1,PRJ d41d8cd98f,Core Deployment,Consultancy,Consultancy,NaN,Billable,EXT c8c92ca432,Unassigned,NaN,Employee,...,Actual,Progressed,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
2,PRJ d41d8cd98f,Core Deployment,Consultancy,Consultancy,NaN,Billable,EXT c8c92ca432,Unassigned,NaN,Employee,...,Actual,Progressed,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
3,PRJ d41d8cd98f,Core Deployment,Consultancy,Consultancy,NaN,Billable,EXT 800982ab08,OB Developer,NaN,Employee,...,Actual,Progressed,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
4,PRJ d41d8cd98f,Core Deployment,Consultancy,Consultancy,NaN,Billable,EXT c8c92ca432,Unassigned,NaN,Employee,...,Actual,Progressed,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0


Next we are going to create a timeseries out of timesheets which is a bit more involved so you can skip it for now and come back to it later, but the points it that at the end you will get a nice monthly pandas Dataframe that looks like this

In [149]:
#collapse

#collapse
dd=[]
timesheet['Period Starting'] = pd.to_datetime(timesheet['Period Starting'])
weekdays = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'] 
for i, weekday in enumerate(weekdays):
    columns = [col for col in timesheet if col not in weekdays or col==weekday]
    tmp = timesheet[columns].copy() #dmytro todo: port backward
    tmp = tmp.rename(columns={weekday: 'effort'})
    tmp['date'] = tmp['Period Starting']+pd.to_timedelta('{}D'.format(i)) #dmytro todo: check period starting offset logic
    tmp['weekday'] = weekday
    tmp = tmp.drop(['Period Starting','Total (Days)'],axis=1)
    dd +=[tmp]

timeseries = pd.concat(dd,sort=False).sort_values(['User','date'])
timeseries = timeseries.reset_index().drop('index',axis=1)
timeseries['date'] = pd.to_datetime(timeseries['date']).astype('str')
timeseries['month'] = timeseries['date'].str[:7] # dmytro todo: maybe improve date, month stylistics 
timeseries['weekend'] = timeseries['weekday'].apply(lambda x: x in ['Sun','Sat'])
timeseries['workweek'] = timeseries['weekday'].apply(lambda x: x not in ['Sun','Sat'])
#timeseries['month']
#dmytro todo: clean almost null rows

In [150]:
timeseries_wide = timeseries.groupby(list(timeseries.columns.drop('effort'))[:2])['effort'].sum()
#.unstack('date').reset_index().fillna(0)
timeseries_wide

Project                              Project Phase                         
A big T&M project                    Delivering the Project                     25.750000
A nice swiss coustomer               Delivering the Project                     43.437500
A pilot for huge opportunity         Billable Travel                             2.250000
                                     Delivering the Project                     71.812500
                                     Overrun on Delivering the Project          21.593750
Another test upgrade                 Overrun on Delivering the Project          15.000000
Big implementation baseline          Delivering the Project                     91.062500
Big implementation rollout           Delivering the Project                     54.312500
                                     Overrun on Delivering the Project           2.000000
                                     Request change                             30.062500
Big upgrade and rework  

In [151]:
timeseries.groupby('User')['effort'].sum()#.unstack('date').reset_index().fillna(0)

User
CATTY THE NEW MANAGER    47.343750
EXT 08a7eee3f6           29.375000
EXT 243a5f992e           20.625000
EXT 29fc29214e           20.875000
EXT 2fbb35eab7           65.600000
EXT 35a1f027d1            0.500000
EXT 35dc2583ea            0.875000
EXT 39cf01a5d7           24.625000
EXT 48e7e5497e           46.000000
EXT 5a04e048b1           57.000000
EXT 5a6cd568b6           20.500000
EXT 5bec8cf95c           18.125000
EXT 6ba130c9d8            3.218750
EXT 6d5bda8573           45.466667
EXT 6ea5e03977           10.460526
EXT 73435c01a6            0.466667
EXT 75da80c2b4            0.250000
EXT 77e3ae3a92           15.500000
EXT 7bdf4bf350           27.000000
EXT 800982ab08           25.750000
EXT 8bf6d9f0ea           15.031250
EXT 8f084cdd2b           63.000000
EXT 8f9ad6f1be           19.937500
EXT 8fcc0898a6            8.250000
EXT 92e6e599a8            3.625000
EXT 95f45ff59d           21.000000
EXT 9666f95122            1.000000
EXT a2155c5e6d           22.066667
EXT a392c18918 

## Profitability

Let me bore you for a minute. You can skip this part if above was self-explanatory to you. Timesheets and projects budgets are simple. You are probably frightened by IFRS revenue recognition and all that – that’s why your CFO has a nice corner office and you don’t, right. Fortunately, there is a simpler thing called Managerial Accounting. 
So, profit is Revenue – Cost and we intend to measure by it and keep it simple. 

**Revenue**: Let’s say that for a Time & Material project you generate as much revenue as you bill (on agreed rate) up to its budget. For a Fixed Fee you split all the revenue per month as per % of effort you spend / are going to spend in the month (changes can be complicated, but they are easy to be managed in the way you want – just go through introduction and wait for the notebook we promised)

In [152]:
timesheet.sum()
revenue_df.effort.sum()
timeseries.effort.sum()

Project             PRJ d41d8cd98fPRJ d41d8cd98fPRJ d41d8cd98fPRJ ...
Project Phase       Core DeploymentCore DeploymentCore DeploymentC...
Work Description    ConsultancyConsultancyConsultancyConsultancyCo...
Activity Type                                                       0
Billing Flag        BillableBillableBillableBillableBillableBillab...
User                EXT 800982ab08EXT c8c92ca432EXT c8c92ca432EXT ...
Roles (Primary)     OB DeveloperUnassignedUnassignedOB DeveloperUn...
Employee Type       EmployeeEmployeeEmployeeEmployeeEmployeeEmploy...
Effort Type         ActualActualActualActualActualReportedActualAc...
Timesheet Status    ProgressedProgressedProgressedProgressedProgre...
Total (Days)                                                  1953.44
Sun                                                           19.9583
Mon                                                           386.355
Tue                                                           401.722
Wed                 

1953.4417763157894

1953.4417763157894

1953.4417763157894

(71, 4)

month
2020-02    813178.333333
2020-03    886836.052632
2020-04    719953.453947
2020-05         0.000000
dtype: float64

In [155]:
# revenue calculation for a person for a month (SUM REV of all timesheet records * corresp project rates)
revenue_df = timeseries[['User','Project','effort','date','month']].merge(projects[['Project','BASE RATE USD']],how='left',on='Project')
revenue_df['BASE RATE USD'] = revenue_df['BASE RATE USD'].fillna(1200)
revenue_df['daily_revenue'] = revenue_df['BASE RATE USD'] * revenue_df['effort']
revenue = revenue_df.groupby(['User','month'])['daily_revenue'].sum().unstack().fillna(0)
revenue = revenue[revenue.sum(1)>0]
revenue_df.effort.sum()
revenue.shape
revenue.sum()
#dmytro todo: check null effort, do smart groupby

1953.4417763157894

(71, 4)

month
2020-02    813178.333333
2020-03    886836.052632
2020-04    719953.453947
2020-05         0.000000
dtype: float64

**Cost**: So, let’s start with the fact that the average cost of $800 (or whatever) is bullshit. You might have people working in different countries and of totally different levels. Talk to your finance team and get some estimates from them (or make it together). We’d say if you were appointed to run a team or business unit – they owe you that much. We will assume below that you managed to get 3 grades (Associate, Base, Senior) per Country you have staff. Though costs per individual will go nicely also.

In [159]:
headcount.head()

,NAME,TYPE,BU,Location,Position,Level,Management size,ID,START
0,JANE DOE,EMPLOYEE,OPS,CH,CON,SEASONED,NO,1001,TBD
1,JOHN DOE,EMPLOYEE,OPS,PL,CON,JUNIOR,NO,1002,TBD
2,JOHN SMITH,EMPLOYEE,OPS,CH,CON,SEASONED,NO,1003,TBD
3,JACK THE MANAGER,CONTRACTOR,OPS,CH,PRO,SENIOR,NO,1004,TBD
4,JOHN THE MANAGER,CONTRACTOR,OPS,CH,PRO,SENIOR,NO,1005,TBD


In [160]:
# cost calculation for a project (SUM COST of all timesheet records * corresp cost rates - see roles)
# dmytro todo: apply this to timeseries

cost_df = timeseries[['User','Project','effort','date','month']].merge(headcount[['NAME','Location','Position','Level']],how='left',left_on='User',right_on='NAME')
cost_df
# cost_df[''] = cost_df['BASE RATE USD'].fillna(1200)
# cost_df['daily_cost'] = cost_df['BASE RATE USD'] * cost_df['effort']
# cost = cost_df.groupby(['User','month'])['daily_cost'].sum().unstack().fillna(0)
# cost = cost[cost.sum(1)>0]
# cost_df.effort.sum()
# cost.shape
# cost.sum()
#dmytro todo: check null effort, do smart groupby

,User,Project,effort,date,month,NAME,Location,Position,Level
0,CATTY THE NEW MANAGER,Upgrade in two contracts - 1,0.0000,2020-02-16,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
1,CATTY THE NEW MANAGER,Upgrade in two contracts - 2,0.0000,2020-02-16,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
2,CATTY THE NEW MANAGER,A pilot for huge opportunity,0.0000,2020-02-16,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
3,CATTY THE NEW MANAGER,A pilot for huge opportunity,0.0000,2020-02-16,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
4,CATTY THE NEW MANAGER,Upgrade in two contracts - 1,0.3750,2020-02-17,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
5,CATTY THE NEW MANAGER,Upgrade in two contracts - 2,0.3125,2020-02-17,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
6,CATTY THE NEW MANAGER,A pilot for huge opportunity,0.3125,2020-02-17,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
7,CATTY THE NEW MANAGER,A pilot for huge opportunity,0.0000,2020-02-17,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
8,CATTY THE NEW MANAGER,Upgrade in two contracts - 1,0.3750,2020-02-18,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED
9,CATTY THE NEW MANAGER,Upgrade in two contracts - 2,0.2500,2020-02-18,2020-02,CATTY THE NEW MANAGER,CH,CON,SEASONED


## More compex stuff

Now let's actually show how to apply data science to data you've seen above

Let's start with a chart on how effort is spent

In [ ]:
# Effort on the project per month

Indeed, you can make it clickable so you can click on the bar and zoom in (it is more then 10 lines, so we will hide the code)

In [ ]:
# (Max) Interactive chart (bar chart effort per month, on click - effort in given month per person)

Ok, as a Team Leader you might not do that. Sorry. Let's get some calculations done. First, let's identify "suspicios" behaviors (e.g. a person who has more then 5 days in a row same effort on a given project and it is not 8 hours) - this is a quick check you do yourself, without asking PMO anything and making this official

In [ ]:
# Simple anomalies detection
# TODO: call for action, intro to next paper

## Use cases

Few cases we identified when this data is helpfull

* Decision making - e.g. identify top loss-making projects
* Identify projects which require management attention
* Identify top performers (profit-wise)
* Identify non-billable hours and internal projects
* Identify suspicious behaviours (we will write another post about it)
    * Employee enters fixed hours per project per day over a period
    * Anomaly detection
* Revenue and effort projections
* Capacity calculations: load all data but plans and generate automated projection (effort * person * month / revenue * month)
* Create consolidated analytic on demand for higher level of management (e.g. profitability forecast, revenue forecast, unallocated capacity) in case your ERP will not do anything like that
    * Example: crisis cost saving - if you decrease hours for some contractors, how much cost do you save vs how much revenue to you lose

But, that's not the point. The point is that loading your own data (excel, csv, tex - whatever) is simple and manipulating it is also simple - definetely simpler then doing it in numerous excel files and faster then waiting for PMOs. 

Below code is to be moved inside and removed

# Grouping data

## setting columns to groupby

In [ ]:
drop_group_columns = ['Total (Days)', 'total_cumsum','Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat','Activity Type','Timesheet Remarks']
groupby_columns = [i for i in timesheet.columns if i not in drop_group_columns]

## one dimensional groupings

In [ ]:
for i in groupby_columns:
  print('\n')
  pd.DataFrame(timesheet.groupby([i])['Total (Days)'].sum()).style


## two dimensional groupbys


In [ ]:
for i in groupby_columns:
  print(i)
  if i != 'Resource Name':
    print('\n')
    df = pd.DataFrame(timesheet.groupby([i,'Resource Name'])['Total (Days)'].sum())#.style
    pd.pivot(df.reset_index(), i ,'Resource Name','Total (Days)').fillna(0)


# Creating timeseries

In [ ]:
pd.to_dattmp['Period Starting']
#+pd.to_timedelta('1D')

In [ ]:
#collapse

#dmytro todo: pretify and apply to new timesheets
dd=[]
cc13 = list(timesheet.columns[:13])
cc12 = [c for c in cc13 if c not in ['Period Starting','Activity Type']]
timesheet['Period Starting'] = pd.to_datetime(timesheet['Period Starting'])
timesheet.sort_values('Period Starting')
for i, weekday in enumerate(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']):
  tmp = timesheet[cc13+[weekday]].copy()
  tmp = tmp.rename(columns={weekday: 'progress'})
  tmp['date'] = tmp['Period Starting']+pd.to_timedelta('{}D'.format(i))
  tmp['weekday'] = weekday
  dd+=[tmp.drop('Period Starting',axis=1)]
timeseries = pd.concat(dd).sort_values(['Resource Name','date'])
timeseries['date']=timeseries['date'].astype('str')
timeseries_workweek = timeseries[~timeseries['weekday'].apply(lambda x: x in ['Sun','Sat']) ].groupby(cc12+['date'])['progress'].sum().unstack('date').reset_index().fillna(0)
timeseries = timeseries.groupby(cc12+['date'])['progress'].sum().unstack('date').reset_index().fillna(0)

## plots with all days of the week

In [ ]:
#figure(figsize=(16,10))
for i in groupby_columns[:]:
  if i not in ['Period Starting']:
    timeseries.groupby([i]).sum().T.cumsum().plot(figsize=(32,20),title=i,grid=True, marker='o', linestyle='--')

## plots with only work days of the week

In [ ]:
#figure(figsize=(16,10))
for i in groupby_columns[:]:
  if i not in ['Period Starting']:
    timeseries_workweek.groupby([i]).sum().T.cumsum().plot(figsize=(32,20),title=i,grid=True, marker='o', linestyle='--')